In [2]:
%%capture
!pip install webdriver-manager
!pip install selenium pandas 

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from pprint import pprint
import random

In [2]:
!pip install selenium

  Using cached selenium-4.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached trio-0.28.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached selenium-4.28.1-py3-none-any.whl (9.5 MB)
Using cached trio-0.28.0-py3-none-any.whl (486 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached attrs-25.1.0-py3-none-any.whl (63 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [5]:
!pip install webdriver-manager

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)
Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl (27 kB)


In [34]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime
import re

def load_cookies(driver, cookie_file):
    try:
        with open(cookie_file, "r") as file:
            cookies = json.load(file)
            if not cookies:
                print("⚠️ Cookie file is empty!")
                return
            
            for cookie in cookies:
                # Convert ISO 8601 expiry to UNIX timestamp
                if "expiry" in cookie:
                    try:
                        cookie["expiry"] = int(datetime.strptime(cookie["expiry"], "%Y-%m-%dT%H:%M:%S.%fZ").timestamp())
                    except ValueError:
                        print(f"⚠️ Skipping cookie {cookie['name']} due to invalid expiry format: {cookie['expiry']}")
                        del cookie["expiry"]  # Remove invalid expiry field
                
                driver.add_cookie(cookie)
            print("✅ Cookies loaded successfully!")
    except json.JSONDecodeError:
        print("⚠️ Error: Invalid JSON format in the cookie file.")
    except Exception as e:
        print(f"⚠️ Error while loading cookies: {e}")

import re

def convert_to_number(value):
    """Convertit les nombres abrégés de Twitter (e.g., '1K', '2.5M') en entiers."""
    value = value.replace(',', '').strip().upper()
    if 'K' in value:
        return int(float(value.replace('K', '')) * 1_000)
    elif 'M' in value:
        return int(float(value.replace('M', '')) * 1_000_000)
    elif 'B' in value:
        return int(float(value.replace('B', '')) * 1_000_000_000)
    else:
        try:
            return int(value)
        except ValueError:
            return 0  # Retourne 0 si la conversion échoue

def extract_last_four_numbers(text):
    """Extrait les 4 derniers nombres du texte et les supprime après extraction."""
    numbers = re.findall(r'\b\d+[KkMmBb]?\b', text)  # Capture les nombres avec K, M ou B
    
    if len(numbers) >= 4:
        replies, retweets, likes, views = numbers[-4:]  # Les 4 dernières valeurs
        
        # Convertir en nombres entiers
        replies = convert_to_number(replies)
        retweets = convert_to_number(retweets)
        likes = convert_to_number(likes)
        views = convert_to_number(views)

        # Supprimer ces valeurs du contenu du tweet
        text = re.sub(r'\b' + re.escape(numbers[-4]) + r'\b', '', text, count=1)
        text = re.sub(r'\b' + re.escape(numbers[-3]) + r'\b', '', text, count=1)
        text = re.sub(r'\b' + re.escape(numbers[-2]) + r'\b', '', text, count=1)
        text = re.sub(r'\b' + re.escape(numbers[-1]) + r'\b', '', text, count=1)
    else:
        replies, retweets, likes, views = 0, 0, 0, 0

    return text.strip(), replies, retweets, likes, views

def extract_tweet_data(tweet):
    """Extrait les données essentielles d'un tweet."""
    try:
        user = tweet.find_element(By.CSS_SELECTOR, 'div[dir="ltr"] span').text
    except:
        user = "Unknown"

    try:
        content = tweet.text.replace("\n", " ")
        clean_content, replies, retweets, likes, views = extract_last_four_numbers(content)
    except:
        content, clean_content = "", ""
        replies, retweets, likes, views = 0, 0, 0, 0

    try:
        date = tweet.find_element(By.CSS_SELECTOR, 'time').get_attribute("datetime")
    except:
        date = "Unknown"

    hashtags = ", ".join(re.findall(r"#\w+", content))
    mentions = ", ".join(re.findall(r"@\w+", content))

    return {
        "User": user,
        "Date": date,
        "Content": clean_content,  # ✅ Contenu nettoyé
        "Hashtags": hashtags,
        "Mentions": mentions,
        "Replies": replies,       # ✅ Maintenant sous forme d'entiers
        "Retweets": retweets,     # ✅ Maintenant sous forme d'entiers
        "Likes": likes,           # ✅ Maintenant sous forme d'entiers
        "Views": views            # ✅ Maintenant sous forme d'entiers
    }


# Function to fetch trending tweets
def fetch_trending_tweets(cookie_file="cookies.json", max_tweets=20):
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36")
    
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.get("https://x.com/home")
    time.sleep(5)

    # Load cookies and refresh
    load_cookies(driver, cookie_file)
    driver.refresh()
    time.sleep(5)

    print("Current page title:", driver.title)  # Debugging login state

    # Verify if cookies are still valid
    print("Final Cookies in Browser:", driver.get_cookies())

    # Go to trending page
    driver.get("https://x.com/search?q=trending&src=trend_click")
    time.sleep(5)

    tweet_data = []
    tweet_count = 0

    while tweet_count < max_tweets:
        tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        print(f"Found {len(tweets)} tweets on this page.")

        for tweet in tweets:
            data = extract_tweet_data(tweet)  # Extract structured tweet data
            tweet_data.append(data)  # Append only the structured data
            tweet_count += 1
            if tweet_count >= max_tweets:
                break

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

    driver.quit()
    return tweet_data

# Run the scraper
tweets = fetch_trending_tweets()

✅ Cookies loaded successfully!
Current page title: Home / X
Final Cookies in Browser: [{'domain': '.x.com', 'expiry': 1773963504, 'httpOnly': False, 'name': 'ct0', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '952037a2f20aaa421f66ffd27f35269fb44773ffcb0d3a4484b340a3d029c9038612f395ab6a82ca74c8bfb704336225e57df63a1ab29815f67467dc953b800c5cbeb015787da35aaabab4c1de63e335'}, {'domain': 'x.com', 'httpOnly': False, 'name': 'lang', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'en'}, {'domain': '.x.com', 'expiry': 1770939506, 'httpOnly': False, 'name': 'twid', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 'u%3D1825171960537513984'}, {'domain': '.x.com', 'expiry': 1773924257, 'httpOnly': True, 'name': 'auth_token', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': '4b37f027aa860025c02899f572d0cb6994f2888f'}, {'domain': '.x.com', 'expiry': 1773963506, 'httpOnly': False, 'name': 'guest_id_ads', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 

In [ ]:
import csv

def save_tweets_to_csv(tweets, output_csv="tweets.csv"):
    if not tweets:
        print("⚠️ No tweets to save!")
        return
    
    # ✅ Correction de l'ordre des colonnes
    fieldnames = ["User", "Date", "Content", "Hashtags", "Mentions", "Replies", "Retweets", "Likes", "Views"]

    with open(output_csv, "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(tweets)

    print(f"✅ {len(tweets)} tweets saved to {output_csv} with structured data!")

# Save the structured tweets
save_tweets_to_csv(tweets)


✅ 20 tweets saved to tweets.csv with structured data!


In [37]:
tweets

[{'User': 'FayeYoko Babebie Taiwan',
  'Date': '2025-02-12T03:32:13.000Z',
  'Content': "FayeYoko Babebie Taiwan @FayeYokoGSTW · 20h We're No.1 on Taiwan's trending list too! Everyone Geng Mak!   SHINE ON FAYEYOKO TO 4E #พาฝ้ายโยโกะคืนให้4E GIF 170 1. .",
  'Hashtags': '#พาฝ',
  'Mentions': '@FayeYokoGSTW',
  'Replies': 3000,
  'Retweets': 2,
  'Likes': 1000,
  'Views': 69000},
 {'User': 'ASHABA',
  'Date': '2025-02-12T17:43:34.000Z',
  'Content': 'ASHABA @sirnghoashaba · 5h Why is she trending??',
  'Hashtags': '',
  'Mentions': '@sirnghoashaba',
  'Replies': 7,
  'Retweets': 2,
  'Likes': 12,
  'Views': 630},
 {'User': 'Mašianoka',
  'Date': '2025-02-11T16:11:22.000Z',
  'Content': 'Mašianoka @Mashianoka_ · Feb 11 i can see why Sophie Rain is trending now… 646   .',
  'Hashtags': '',
  'Mentions': '@Mashianoka_',
  'Replies': 466,
  'Retweets': 18000,
  'Likes': 1,
  'Views': 7000000},
 {'User': '𝐍𝐚𝐯𝐞𝐞𝐧 𝐘𝐒𝐉 𝐕𝐢𝐳𝐚𝐠',
  'Date': '2025-02-11T14:18:23.000Z',
  'Content': "𝐍𝐚𝐯𝐞𝐞𝐧 𝐘𝐒𝐉 𝐕𝐢𝐳𝐚𝐠 